In [4]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [5]:
(X_train_full,y_train_full),(X_test,y_test)=tf.keras.datasets.mnist.load_data()
X_train_full=X_train_full/255.
X_test=X_test/255.
X_valid,X_train=X_train_full[:5000],X_train_full[5000:]
y_valid,y_train=y_train_full[:5000],y_train_full[5000:]

11501568/11490434 [==============================] - 0s 0us/step


In [6]:
tf.random.set_seed(42)
np.random.seed(42)
LAYERS=[tf.keras.layers.Flatten(input_shape=[28,28]),
         tf.keras.layers.Dense(300,kernel_initializer='he_normal') ,
         tf.keras.layers.LeakyReLU(),
         tf.keras.layers.Dense(100,kernel_initializer='he_normal') ,
            tf.keras.layers.LeakyReLU(),
            tf.keras.layers.Dense(10,activation="softmax")  ]
model=tf.keras.models.Sequential(LAYERS)

In [7]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),metrics=["accuracy"])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [9]:
history=model.fit(X_train,y_train,epochs=10,validation_data=(X_valid,y_valid),verbose=2)

Epoch 1/10
1719/1719 - 6s - loss: 1.5275 - accuracy: 0.5970 - val_loss: 0.9444 - val_accuracy: 0.7980 - 6s/epoch - 3ms/step
Epoch 2/10
1719/1719 - 5s - loss: 0.7465 - accuracy: 0.8287 - val_loss: 0.5868 - val_accuracy: 0.8596 - 5s/epoch - 3ms/step
Epoch 3/10
1719/1719 - 5s - loss: 0.5412 - accuracy: 0.8624 - val_loss: 0.4685 - val_accuracy: 0.8834 - 5s/epoch - 3ms/step
Epoch 4/10
1719/1719 - 5s - loss: 0.4591 - accuracy: 0.8771 - val_loss: 0.4104 - val_accuracy: 0.8940 - 5s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 5s - loss: 0.4142 - accuracy: 0.8869 - val_loss: 0.3758 - val_accuracy: 0.9006 - 5s/epoch - 3ms/step
Epoch 6/10
1719/1719 - 5s - loss: 0.3852 - accuracy: 0.8938 - val_loss: 0.3525 - val_accuracy: 0.9052 - 5s/epoch - 3ms/step
Epoch 7/10
1719/1719 - 5s - loss: 0.3644 - accuracy: 0.8980 - val_loss: 0.3348 - val_accuracy: 0.9102 - 5s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 5s - loss: 0.3485 - accuracy: 0.9021 - val_loss: 0.3209 - val_accuracy: 0.9138 - 5s/epoch - 3ms/step
Epoch 9/

In [10]:
model.save("pre_mnist_model.h5")

In [11]:
pretrained_mnist_model=tf.keras.models.load_model("pre_mnist_model.h5")

In [12]:
pretrained_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [13]:
for layers in pretrained_mnist_model.layers:
  print(f"{layers.name}:{layers.trainable}")

flatten:True
dense:True
leaky_re_lu:True
dense_1:True
leaky_re_lu_1:True
dense_2:True


In [14]:
for layers in pretrained_mnist_model.layers[:-1]:
  layers.trainable=False
  print(f"{layers.name}:{layers.trainable}")

flatten:False
dense:False
leaky_re_lu:False
dense_1:False
leaky_re_lu_1:False


In [15]:
for layers in pretrained_mnist_model.layers:
  print(f"{layers.name}:{layers.trainable}")

flatten:False
dense:False
leaky_re_lu:False
dense_1:False
leaky_re_lu_1:False
dense_2:True


In [16]:
lower_pretrained_model=pretrained_mnist_model.layers[:-1]
new_model=tf.keras.models.Sequential(lower_pretrained_model)

In [17]:
new_model.add(tf.keras.layers.Dense(2,activation="softmax"),)

In [18]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 202       
                                                                 
Total params: 265,802
Trainable params: 202
Non-traina

In [19]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [20]:
np.where(y_train%2==0,1,0)

array([0, 0, 1, ..., 0, 1, 1])

In [21]:
y_train[0],y_train[-1]

(7, 8)

In [22]:
def update_even_odd_labels(labels):
  for idx,label in enumerate(labels):
    labels[idx]=np.where(label%2==0,1,0)
  return labels

In [23]:
y_train_bin,y_test_bin,y_valid_bin=update_even_odd_labels([y_train,y_test,y_valid])

In [24]:
np.unique(y_train_bin),np.unique(y_test_bin),np.unique(y_valid_bin)

(array([0, 1]), array([0, 1]), array([0, 1]))

In [25]:

new_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),metrics=["accuracy"])

In [26]:
new_history=new_model.fit(X_train,y_train_bin,epochs=10,validation_data=(X_valid,y_valid_bin),verbose=2)

Epoch 1/10
1719/1719 - 4s - loss: 0.3898 - accuracy: 0.8288 - val_loss: 0.3247 - val_accuracy: 0.8676 - 4s/epoch - 2ms/step
Epoch 2/10
1719/1719 - 3s - loss: 0.3300 - accuracy: 0.8602 - val_loss: 0.3049 - val_accuracy: 0.8752 - 3s/epoch - 2ms/step
Epoch 3/10
1719/1719 - 3s - loss: 0.3163 - accuracy: 0.8660 - val_loss: 0.2948 - val_accuracy: 0.8796 - 3s/epoch - 2ms/step
Epoch 4/10
1719/1719 - 3s - loss: 0.3083 - accuracy: 0.8701 - val_loss: 0.2884 - val_accuracy: 0.8832 - 3s/epoch - 2ms/step
Epoch 5/10
1719/1719 - 3s - loss: 0.3023 - accuracy: 0.8725 - val_loss: 0.2834 - val_accuracy: 0.8846 - 3s/epoch - 2ms/step
Epoch 6/10
1719/1719 - 3s - loss: 0.2978 - accuracy: 0.8752 - val_loss: 0.2792 - val_accuracy: 0.8874 - 3s/epoch - 2ms/step
Epoch 7/10
1719/1719 - 3s - loss: 0.2939 - accuracy: 0.8772 - val_loss: 0.2758 - val_accuracy: 0.8872 - 3s/epoch - 2ms/step
Epoch 8/10
1719/1719 - 3s - loss: 0.2907 - accuracy: 0.8788 - val_loss: 0.2728 - val_accuracy: 0.8890 - 3s/epoch - 2ms/step
Epoch 9/

In [27]:
new_model.evaluate(X_test,y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.2768 - accuracy: 0.8861


[0.2768312990665436, 0.8860999941825867]

In [28]:
X_new=X_test[:3]
y_test[:3],y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([0, 1, 0]))

In [29]:
np.argmax(new_model.predict(X_new),axis=-1)

array([0, 1, 0])

label----------------------------

In [31]:
def update_even_odd_labels(labels):
  for idx,label in enumerate(labels):
    labels[idx]=np.where(label>5,1,0)
  return labels
  

In [32]:
y_train_bin,y_test_bin,y_valid_bin=update_even_odd_labels([y_train,y_test,y_valid])

In [33]:
np.unique(y_train_bin),np.unique(y_test_bin),np.unique(y_valid_bin)

(array([0, 1]), array([0, 1]), array([0, 1]))

In [34]:

new_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),metrics=["accuracy"])

In [35]:
new_history=new_model.fit(X_train,y_train_bin,epochs=10,validation_data=(X_valid,y_valid_bin),verbose=2)

Epoch 1/10
1719/1719 - 3s - loss: 0.6059 - accuracy: 0.7084 - val_loss: 0.4501 - val_accuracy: 0.7822 - 3s/epoch - 2ms/step
Epoch 2/10
1719/1719 - 3s - loss: 0.4172 - accuracy: 0.8163 - val_loss: 0.3871 - val_accuracy: 0.8394 - 3s/epoch - 2ms/step
Epoch 3/10
1719/1719 - 3s - loss: 0.3813 - accuracy: 0.8428 - val_loss: 0.3611 - val_accuracy: 0.8556 - 3s/epoch - 2ms/step
Epoch 4/10
1719/1719 - 3s - loss: 0.3635 - accuracy: 0.8536 - val_loss: 0.3471 - val_accuracy: 0.8628 - 3s/epoch - 2ms/step
Epoch 5/10
1719/1719 - 3s - loss: 0.3520 - accuracy: 0.8596 - val_loss: 0.3352 - val_accuracy: 0.8642 - 3s/epoch - 2ms/step
Epoch 6/10
1719/1719 - 3s - loss: 0.3435 - accuracy: 0.8638 - val_loss: 0.3279 - val_accuracy: 0.8674 - 3s/epoch - 2ms/step
Epoch 7/10
1719/1719 - 3s - loss: 0.3370 - accuracy: 0.8662 - val_loss: 0.3208 - val_accuracy: 0.8728 - 3s/epoch - 2ms/step
Epoch 8/10
1719/1719 - 3s - loss: 0.3315 - accuracy: 0.8697 - val_loss: 0.3159 - val_accuracy: 0.8746 - 3s/epoch - 2ms/step
Epoch 9/

In [36]:
new_model.evaluate(X_test,y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.3133 - accuracy: 0.8789


[0.31329450011253357, 0.8788999915122986]

In [37]:
X_new=X_test[:3]
y_test[:3],y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([1, 0, 0]))

In [38]:
np.argmax(new_model.predict(X_new),axis=-1)

array([1, 0, 0])